In [1]:
require "hdf5"
require "optim"
require "nn"

In [2]:
f = hdf5.open("data.hdf5", "r")

X_train = f:read("X_train"):all()
Y_train = f:read("Y_train"):all()
X_valid = f:read("X_valid"):all()
Y_valid = f:read("Y_valid"):all()
X_test = f:read("X_test"):all()
nwords = f:read("nwords"):all()[1]
nclasses = f:read("nclasses"):all()[1]
nfeatures = f:read("nfeatures"):all()[1]

--sentences
X_valid_sen = f:read("X_valid_sen"):all()
X_test_sen = f:read("X_test_sen"):all()

In [ ]:
--C_emi = torch.zeros(nclasses, nwords, nfeatures)

In [ ]:
function fit(X, Y, alpha1, alpha2)
    C_trans = torch.ones(nclasses,nclasses)*alpha1
    C_emi = torch.ones(nclasses, nwords)*alpha2
    for i = 2,X:size(1) do
        y_curr = Y[i]
        y_prev = Y[i-1]
        x_curr = X[i]
        C_trans[y_prev][y_curr] = C_trans[y_prev][y_curr] + 1
        C_emi[y_curr][x_curr] = C_emi[y_curr][x_curr] + 1
    end
    return C_trans, C_emi
end

function distribution(y_prev, x_curr, C_trans, C_emi)
--compute 
    trans = C_trans[y_prev]/torch.sum(C_trans[y_prev])
    emi = C_emi[{{},{x_curr,x_curr}}]/torch.sum(C_emi[{{},{x_curr,x_curr}}])
    return torch.log(trans) + torch.log(emi)
end

In [ ]:
--fits the count-based model with given smoothing parameters
ct, ce = fit(X_train, Y_train, 1, 1)

In [ ]:
--returns a distribution over the various tags given the previous tag, the current feature, and the count matrices
p = distribution(2, 15, ct, ce)